<a href="https://colab.research.google.com/github/CharlieKerfoot/finance-agent/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install weave wandb
!pip install --no-deps xformer trl peft accelerate bitsandbytes

In [ ]:
import wandb
wandb.login()

In [ ]:
import weave

run = wandb.init(
    project="Arb-Agent-v1",
    name="llama3-8b-lora-run3",
    job_type="training",
)

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 8192
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ckerf/arb-instruct-v1", split="train")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def format_prompts(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
      text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
      texts.append(text)
    return {
      "text": texts,
    }

dataset = dataset.map(format_prompts, batched=True)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 8192,
    dataset_num_proc = 8,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        num_train_epochs=1,
        warmup_steps = 10,
        learning_rate = 2e-4,
        fp16 = False,
        bf16 = True,
        logging_steps = 25,
        logging_strategy = "steps",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        report_to = "wandb",
        run_name = "llama3-8b-lora-run2",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
FastLanguageModel.for_inference(model)

import random
samples = random.sample(list(dataset), 5)

for sample in samples:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                sample["instruction"],
                sample["input"],
                "",
            )
        ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)

    print("\n" + "-" * 50 + "\n")
    print(f"Q: {sample['instruction']}")
    print(f"A:\n{tokenizer.batch_decode(outputs)[0].split('### Response:')[1]}")
    print(f"Ground Truth:\n{sample['output']}")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("ckerf/arbagent-llama3-8b-lora")
tokenizer.push_to_hub("ckerf/arbagent-llama3-8b-lora")